<a href="https://colab.research.google.com/github/Mary7Magdalene/MLL/blob/main/fake_news_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependancies

In [39]:
import numpy as np
import pandas as pd
import re #for search purposes
from nltk.corpus import stopwords # used to extract the unimportant words
from nltk.stem.porter import PorterStemmer # give the root words
from sklearn.feature_extraction.text import TfidfVectorizer #for converting text into numbers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data preprocessing

In [5]:
#loading the dataset into  a pandas dataframe
news_dataset = pd.read_csv('/content/train.csv')

In [6]:
news_dataset.shape

(20800, 5)

In [7]:
#printing the first five rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
news_dataset.isnull().sum()

,0
id,0
title,558
author,1957
text,39
label,0


In [9]:
#replacing the null values with an empty string
news_dataset = news_dataset.fillna('')

In [10]:
#merging the author name and news title
news_dataset['content'] = news_dataset['author'] +''+news_dataset['title']

In [11]:
print(news_dataset['content'])

0        Darrell LucusHouse Dem Aide: We Didn’t Even Se...
1        Daniel J. FlynnFLYNN: Hillary Clinton, Big Wom...
2        Consortiumnews.comWhy the Truth Might Get You ...
3        Jessica Purkiss15 Civilians Killed In Single U...
4        Howard PortnoyIranian woman jailed for fiction...
                               ...                        
20795    Jerome HudsonRapper T.I.: Trump a ’Poster Chil...
20796    Benjamin HoffmanN.F.L. Playoffs: Schedule, Mat...
20797    Michael J. de la Merced and Rachel AbramsMacy’...
20798    Alex AnsaryNATO, Russia To Hold Parallel Exerc...
20799               David SwansonWhat Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [12]:
#separating the data and labels
x = news_dataset.drop(columns ='label', axis =1)
y = news_dataset['label']

In [13]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

STEMMING

stemming is the process of reducing a word to its root wo[link text](https://)rd

In [14]:
port_stem = PorterStemmer()

In [15]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]','',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [16]:
news_dataset['content ']= news_dataset['content'].apply(stemming)

In [17]:
print(news_dataset['content '])

0        darrelllucushousedemaidewedidntevenseecomeysle...
1        danieljflynnflynnhillaryclintonbigwomanoncampu...
2               consortiumnewscomwhythetruthmightgetyoufir
3        jessicapurkisscivilianskilledinsingleusairstri...
4        howardportnoyiranianwomanjailedforfictionalunp...
                               ...                        
20795    jeromehudsonrappertitrumpaposterchildforwhites...
20796    benjaminhoffmannflplayoffsschedulematchupsando...
20797    michaeljdelamercedandrachelabramsmacysissaidto...
20798    alexansarynatorussiatoholdparallelexercisesinb...
20799                          davidswansonwhatkeepsthefal
Name: content , Length: 20800, dtype: object


In [18]:
#separating the label and the data
x = news_dataset['content'].values
y = news_dataset['label'].values

In [19]:
print(x)

['Darrell LucusHouse Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'
 'Daniel J. FlynnFLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'
 'Consortiumnews.comWhy the Truth Might Get You Fired' ...
 'Michael J. de la Merced and Rachel AbramsMacy’s Is Said to Receive Takeover Approach by Hudson’s Bay - The New York Times'
 'Alex AnsaryNATO, Russia To Hold Parallel Exercises In Balkans'
 'David SwansonWhat Keeps the F-35 Alive']


In [20]:
print(y)

[1 0 1 ... 0 1 1]


In [21]:
y.shape


(20800,)

In [22]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x,y)

x = vectorizer.transform(x)

In [23]:
print(x)

  (0, 1181)	0.26093253981852516
  (0, 5558)	0.3015240959717515
  (0, 6517)	0.20565002646673405
  (0, 7840)	0.2951614509491496
  (0, 8241)	0.2740085657861192
  (0, 8620)	0.24398932179297475
  (0, 10594)	0.23028973074454998
  (0, 16295)	0.1536111400920685
  (0, 16508)	0.20476648736259792
  (0, 18475)	0.24826788911182338
  (0, 19040)	0.35769014842767877
  (0, 28544)	0.22370120836039492
  (0, 33092)	0.34354025525372645
  (0, 33460)	0.2620405036866357
  (0, 34402)	0.17918739747269477
  (1, 3621)	0.3493832273397924
  (1, 4420)	0.18290583060889654
  (1, 5116)	0.4508844821928066
  (1, 6219)	0.23118052033472167
  (1, 7777)	0.313608220062674
  (1, 11652)	0.5156618392052476
  (1, 14631)	0.23703745360274664
  (1, 22730)	0.19046856538729842
  (1, 35080)	0.3619575823267221
  (2, 6696)	0.5074493780278307
  :	:
  (20797, 20519)	0.16984739136130164
  (20797, 21898)	0.07697953798217144
  (20797, 25706)	0.23832116836576178
  (20797, 26083)	0.30196639491652494
  (20797, 27797)	0.23692575764592924
  (20797

In [24]:
#splitting the dataset into rtraining and test data
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, stratify = y, random_state =2)

In [25]:
print(x.shape, x_train.shape, x_test.shape)

(20800, 36678) (16640, 36678) (4160, 36678)


Training the model

In [26]:
model = LogisticRegression()

In [27]:
model.fit(x_train,y_train)

LogisticRegression()

Evaluating the model

In [28]:
#Accuracy on the training data
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction, y_train)

In [29]:
print(training_data_accuracy)

0.9716346153846154


In [30]:
#accuracy on test data
x_test_prediction = model.predict(x_test)
test_data_accuracy = accuracy_score(x_test_prediction, y_test)

In [31]:
print(test_data_accuracy)

0.9548076923076924


Building a predictive system

In [37]:
x_new = x_test[3]

prediction = model.predict(x_new)
print(prediction)

if (prediction[0] == 0):
  print('The news is real')
else:
  print('The news is fake')


[0]
The news is real


In [38]:
print(y_test[3])

0
